# Training the LDA model on the tweets from the bot challenge

In [11]:
# Imports
import numpy as np
import pandas as pd

# So ridicolous that this is the recommended approach
# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
import os
import sys
# add the 'dashboard' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'dashboard')
sys.path.append(src_dir)

# import my method from the source code
%aimport streaming.mongo_consumer
from streaming.mongo_consumer import MongoConsumer

from pyspark.sql import SparkSession, SQLContext
from pyspark.streaming import StreamingContext

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

# Load and parse the data
data = sc.textFile("../data/processed/tweets_text.txt")





parsedData = data.map(lambda line: Vectors.dense([x for x in line.strip().split(' ')]))

"""
Load each file as an individual document
Strip any leading or trailing whitespace
Convert all characters into lowercase where applicable
Split each document into words, separated by whitespace, semi-colons, commas, and octothorpes
Only keep the words that are all alphabetical characters
Only keep words larger than 3 characters
http://sean.lane.sh/blog/2016/PySpark_and_LDA
"""
corpus = data                                          \
    .map( lambda document: document.strip().lower())            \
    .map( lambda document: re.split("[\s;,#]", document))       \
    .map( lambda word: [x for x in word if x.isalpha()])        \
    .map( lambda word: [x for x in word if len(x) > 3] )        \
    .cache()
    

# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3)

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
      + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

# Save model
ldaModel.save(sc, "../models/lda")
#sameModel = LDAModel\
#    .load(sc, "../models/lda")


Py4JJavaError: An error occurred while calling o138.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 4, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/claasmeiners/Library/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/Users/claasmeiners/Library/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/claasmeiners/Library/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-5-eee4e4cae1bd>", line 13, in <lambda>
NameError: name 're' is not defined

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.P

## Lets work with 10,000 Users to begin with

For more we need to switch to batched/streamed/cursored approach as not all tweets can be loaded into memory